# KAMPING Tutorial 2. Homogenous Graph neural network modeling

Date created: 2024-10-25

In [2]:
# Import kamping library before starting the tutorial
import kamping

%load_ext autoreload
%autoreload 2

flak## 1. Create a list of KeggGraph objects from a directory with KGML files

In the previous tutorial we have shown how to use parse information from a single KGML file into KeggGraph object for storing information in a easily-access way. In this tutorial, we will show you how to dataset can be used in one of the most popular graph-machine learning package "pytorch-geometric" through provided utility function with ease.

Machine-learning graph model also use data contains more than one graphs, you can use `kamping.create_graphs` function to create a list of KeggGraph objects from a directory with KGML files.

In this tutorial we target the homogenous graph, which is defined only have one type of nodes in a graph. In our case, a homogenous graph is a "gene-only" graph or "metbaolite-only" graph. Training on Homogenous graph is easy to understand, which is the reason why we start from here. Later, we will show you KAMPING can also convert heterogenous graph in a similar way with just a littble bit extra effort. 

In [3]:
gene_graphs = kamping.create_graphs('../data/kgml_hsa', type='gene', verbose=True, ignore_file=['hsa01100.xml'])


            Visit https://www.kegg.jp/kegg-bin/show_pathway?hsa00190 for pathway details.

            There are likely no edges in which to parse...
INFO:KeggGraph:Now parsing: path:hsa00220...
INFO:KeggGraph:Graph path:hsa00220 parsed successfully!
INFO:KeggGraph:Now parsing: path:hsa00230...
INFO:KeggGraph:Graph path:hsa00230 parsed successfully!
INFO:KeggGraph:Now parsing: path:hsa00232...
INFO:KeggGraph:Graph path:hsa00232 parsed successfully!
INFO:KeggGraph:Now parsing: path:hsa00240...
INFO:KeggGraph:Graph path:hsa00240 parsed successfully!
INFO:KeggGraph:Now parsing: path:hsa00250...
INFO:KeggGraph:Graph path:hsa00250 parsed successfully!
INFO:KeggGraph:Now parsing: path:hsa00260...
INFO:KeggGraph:Graph path:hsa00260 parsed successfully!
INFO:KeggGraph:Now parsing: path:hsa00270...
INFO:KeggGraph:Graph path:hsa00270 parsed successfully!
INFO:KeggGraph:Now parsing: path:hsa00280...
INFO:KeggGraph:Graph path:hsa00280 parsed successfully!
INFO:KeggGraph:Now parsing: path:hsa00290

The batch processing of KGML could also be useful in regular task. To access the result of specific KGML file you can use code below.  

In [4]:
gene_graph_00010 = [graph for graph in gene_graphs if graph.name == 'path:hsa00010'][0]

In [5]:
gene_graph_00010

KEGG Pathway: 
            [Title]: Glycolysis / Gluconeogenesis
            [Name]: path:hsa00010
            [Org]: hsa
            [Link]: https://www.kegg.jp/kegg-bin/show_pathway?hsa00010
            [Image]: https://www.kegg.jp/kegg/pathway/hsa/hsa00010.png
            [Link]: https://www.kegg.jp/kegg-bin/show_pathway?hsa00010
            Graph type: gene 
            Number of Genes: 67
            Number of Compounds: 0
            Gene ID type : kegg
            Compound ID type : kegg
            Number of Nodes: 67
            Number of Edges: 559

In [6]:
gene_graph_00010.edges

,entry1,entry2,type,subtype_name,subtype_value,entry1_type,entry2_type
0,hsa:10327,hsa:124,PPrel,compound-propagation,custom,gene,gene
1,hsa:10327,hsa:125,PPrel,compound-propagation,custom,gene,gene
2,hsa:10327,hsa:126,PPrel,compound-propagation,custom,gene,gene
3,hsa:10327,hsa:127,PPrel,compound-propagation,custom,gene,gene
4,hsa:10327,hsa:128,PPrel,compound-propagation,custom,gene,gene
...,...,...,...,...,...,...,...
554,hsa:9562,hsa:387712,PPrel,compound-propagation,custom,gene,gene
555,hsa:9562,hsa:441531,PPrel,compound-propagation,custom,gene,gene
556,hsa:9562,hsa:5223,PPrel,compound-propagation,custom,gene,gene
557,hsa:9562,hsa:5224,PPrel,compound-propagation,custom,gene,gene


In this tutorial, we will use pre-processed protein embedding information directly from uniprot, so we need to convert the KEGG gene ID into UniProt ID. We don't need to convert the KEGG compound id so we keep it untouched. If you didn't specify the "compound_target" when initalizing the converter, it will be default as "kegg". The same if you only want to convert gene ID. 

In [7]:
converter = kamping.Converter('hsa', gene_target='uniprot', verbose=True)

In [8]:
for graph in gene_graphs:
    converter.convert(graph)

INFO:kamping.parser.convert:Conversion of path:hsa00010 complete!
INFO:kamping.parser.convert:Conversion of path:hsa00020 complete!
INFO:kamping.parser.convert:Conversion of path:hsa00030 complete!
INFO:kamping.parser.convert:Conversion of path:hsa00040 complete!
INFO:kamping.parser.convert:Conversion of path:hsa00051 complete!
INFO:kamping.parser.convert:Conversion of path:hsa00052 complete!
INFO:kamping.parser.convert:Conversion of path:hsa00053 complete!
INFO:kamping.parser.convert:Conversion of path:hsa00061 complete!
INFO:kamping.parser.convert:Conversion of path:hsa00062 complete!
INFO:kamping.parser.convert:Conversion of path:hsa00071 complete!
INFO:kamping.parser.convert:Conversion of path:hsa00100 complete!
INFO:kamping.parser.convert:Conversion of path:hsa00120 complete!
INFO:kamping.parser.convert:Conversion of path:hsa00130 complete!
INFO:kamping.parser.convert:Conversion of path:hsa00140 complete!
INFO:kamping.parser.convert:Conversion of path:hsa00220 complete!
INFO:kampi

If you didn't convert Compound ID into other ID. You can use `kamping.get_kegg_mol` function to retrieve the molfile from KEGG for each compound in all graphs and create a MOL object using RDKit (https://www.rdkit.org/). It will return a pd.dataframe with first column the compound ID and second column as the MOL object.

In [9]:
# uncomment to run for the first time
# mols = kamping.get_kegg_mol(graphs)

The process might take a while due to the large number of compounds from so many graphs. It could a good idea to save the created pd.DataFrame for repeated use when testing different approach of embedding metabolite.

In [10]:
import pandas as pd

# uncommented code below if run the first time
# save the mols to a file
# mols.to_pickle('data/mols.pkl')
# retrieve mol from file
mols = pd.read_pickle('data/mols.pkl')

Not all compound has a molFile from KEGG. Most compounds without molFile are glycan which is doesn't have a fixed atom composition.  Right now, we can just ignore them.

In [11]:
mols

,id,ROMol
0,cpd:C00038,<rdkit.Chem.rdchem.Mol object at 0x2d1da7150>
1,cpd:C01180,<rdkit.Chem.rdchem.Mol object at 0x2d1bde890>
2,gl:G00083,None
3,cpd:C20683,<rdkit.Chem.rdchem.Mol object at 0x2d1bdeed0>
4,cpd:C02593,<rdkit.Chem.rdchem.Mol object at 0x2d1bdd260>
...,...,...
1658,gl:G10599,None
1659,cpd:C03090,<rdkit.Chem.rdchem.Mol object at 0x2d20ced90>
1660,cpd:C00097,<rdkit.Chem.rdchem.Mol object at 0x2d20ced40>
1661,cpd:C11134,<rdkit.Chem.rdchem.Mol object at 0x2cc97ecf0>


After we get the MOL object of each compound, we can use RDkit to embedding them into vectors that can be understanded by machine.

In [12]:
# todo: Might be a good idea to depend on scikit-mol  

In [13]:
mol_embeddings = kamping.get_mol_embeddings_from_dataframe(mols, transformer='morgan')

'
                    total 231 Invalid rows with "None" in the ROMol column


In [14]:
protein_embeddings = kamping.get_uniprot_protein_embeddings(gene_graphs, '../data/embedding/protein_embedding.h5')

In [15]:
gene_graphs[0]

KEGG Pathway: 
            [Title]: Glycolysis / Gluconeogenesis
            [Name]: path:hsa00010
            [Org]: hsa
            [Link]: https://www.kegg.jp/kegg-bin/show_pathway?hsa00010
            [Image]: https://www.kegg.jp/kegg/pathway/hsa/hsa00010.png
            [Link]: https://www.kegg.jp/kegg-bin/show_pathway?hsa00010
            Graph type: gene 
            Number of Genes: 104
            Number of Compounds: 0
            Gene ID type : uniprot
            Compound ID type : kegg
            Number of Nodes: 104
            Number of Edges: 1278

## 2. Create a Pytorch-geometric data object

In [90]:
pyg_one_graph, mapping = kamping.convert_to_single_pyg(gene_graphs, embeddings=protein_embeddings)

In [91]:
data = pyg_one_graph
data

Data(x=[7275, 1024], edge_index=[2, 108732])

In [92]:
original_edge_index = data.edge_index

Pytorch-geometric data mainly consist of "edge_index" and "x", which is the feature of nodes. Other information are also saved for prediction interpretation, such as the node_type, node original name ("node_name"), "edge_type", "edge_subtype_name". Other information such as "name" is "combined" indicate it  is a combined graph from small graphs and type="gene" indicate it is a homogenous "gene" graph.

In [46]:
from torch_geometric.loader import DataLoader
dataloader = DataLoader([data])
batch = next(iter(dataloader))

## 3. Training graph neural network model

In [47]:
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv
from torch_geometric.utils import train_test_split_edges

In [48]:
data

Data(x=[7275, 1024], edge_index=[2, 108732])

In [49]:
import torch_geometric.transforms as T
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    is_undirected=True,
    neg_sampling_ratio=1.0,
    add_negative_train_samples=False
)
train_data, val_data, test_data = transform(data)

In [57]:
train_data

Data(x=[7275, 1024], edge_index=[2, 80734], edge_label=[40367], edge_label_index=[2, 40367])

In [50]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

In [78]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(1024, 128)
        self.conv2 = SAGEConv(128, 64)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index) # convolution 1
        x = x.relu()
        return self.conv2(x, edge_index) # convolution 2

    def decode(self, z, pos_edge_index, neg_edge_index): # only pos and neg edges
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1) # concatenate pos and neg edges
        logits = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)  # dot product 
        return logits

    def decode_all(self, z):
        prob_adj = z @ z.t() # get adj NxN
        return (prob_adj > 0).nonzero(as_tuple=False).t() # get predicted edge_list 

In [79]:
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

In [84]:
import torch_geometric.utils as utils

def get_link_labels(pos_edge_index, neg_edge_index):
    # returns a tensor:
    # [1,1,1,1,...,0,0,0,0,0,..] with the number of ones is equel to the lenght of pos_edge_index
    # and the number of zeros is equal to the length of neg_edge_index
    E = pos_edge_index.size(1) + neg_edge_index.size(1)
    link_labels = torch.zeros(E, dtype=torch.float, device=device)
    link_labels[:pos_edge_index.size(1)] = 1.
    return link_labels

def train(model, data):
    model.train()

    pos_edge_label_index = data.edge_label_index
    pos_edge_label = torch.ones(pos_edge_label_index.size(1))


    neg_edge_label_index = utils.negative_sampling(edge_index=pos_edge_label_index, #positive edges
                                                   num_nodes=data.x.size(0), # number of nodes
                                                   num_neg_samples=pos_edge_label_index.size(1))
    neg_edge_label = torch.zeros(neg_edge_label_index.size(1))

    optimizer.zero_grad()

    z = model(data.x, pos_edge_label_index) #encode
    link_logits = model.decode(z, pos_edge_label_index, neg_edge_label_index) # decode

    link_labels = get_link_labels(pos_edge_label_index, neg_edge_label_index)
    loss = F.binary_cross_entropy_with_logits(link_logits, link_labels)
    loss.backward()
    optimizer.step()

    return loss

In [85]:
@torch.no_grad()
def validate(model, data):
    z = model(data.x, data.edge_index)
    edge_label_index = data.edge_label_index
    edge_label = data.edge_label
    z_src = z[edge_label_index[0]]
    z_dst = z[edge_label_index[1]]

    recon = (z_src * z_dst).sum(dim=-1)
    loss = F.binary_cross_entropy_with_logits(recon, edge_label)
    # calculate AUC
    auc = roc_auc_score(edge_label.cpu().detach().numpy(), recon.cpu().detach().numpy())

    return loss.item(), auc


In [86]:
@torch.no_grad()
def test(model, data):
    z = model(data.x, data.edge_index)
    edge_label_index = data.edge_label_index
    edge_label = data.edge_label
    z_src = z[edge_label_index[0]]
    z_dst = z[edge_label_index[1]]

    recon = (z_src * z_dst).sum(dim=-1)
    loss = F.binary_cross_entropy_with_logits(recon, edge_label)
    # calculate AUC
    auc = roc_auc_score(edge_label.cpu().detach().numpy(), recon.cpu().detach().numpy())

    return loss.item(), auc


In [87]:
for epoch in range(1, 1001):
    train_loss = train(model, train_data)
    val_loss, val_auc= test(model, val_data)
    test_loss, test_auc = test(model, test_data)
    print(f'Epoch: {epoch:03d}, Loss: {train_loss:.4f}, AUC Val: {val_auc:.4f}, AUC Test: {test_auc:.4f}')

Epoch: 001, Loss: 0.6958, AUC Val: 0.7375, AUC Test: 0.7345
Epoch: 002, Loss: 1.6120, AUC Val: 0.6724, AUC Test: 0.6557
Epoch: 003, Loss: 0.7194, AUC Val: 0.6837, AUC Test: 0.6751
Epoch: 004, Loss: 0.7185, AUC Val: 0.7313, AUC Test: 0.7259
Epoch: 005, Loss: 0.6827, AUC Val: 0.7359, AUC Test: 0.7341
Epoch: 006, Loss: 0.6713, AUC Val: 0.7230, AUC Test: 0.7235
Epoch: 007, Loss: 0.6621, AUC Val: 0.7198, AUC Test: 0.7208
Epoch: 008, Loss: 0.6591, AUC Val: 0.7240, AUC Test: 0.7254
Epoch: 009, Loss: 0.6501, AUC Val: 0.7331, AUC Test: 0.7344
Epoch: 010, Loss: 0.6401, AUC Val: 0.7444, AUC Test: 0.7448
Epoch: 011, Loss: 0.6322, AUC Val: 0.7541, AUC Test: 0.7550
Epoch: 012, Loss: 0.6218, AUC Val: 0.7634, AUC Test: 0.7642
Epoch: 013, Loss: 0.6175, AUC Val: 0.7768, AUC Test: 0.7796
Epoch: 014, Loss: 0.6091, AUC Val: 0.7902, AUC Test: 0.7939
Epoch: 015, Loss: 0.6000, AUC Val: 0.7989, AUC Test: 0.8024
Epoch: 016, Loss: 0.5907, AUC Val: 0.8022, AUC Test: 0.8054
Epoch: 017, Loss: 0.5789, AUC Val: 0.806

KeyboardInterrupt: 

# Save data and load data for later use

# prediction


In [ ]:
z = model.encode() #encode
predict_adj = torch.sigmoid(z @ z.t())
# get 2D index of the top 10% edges
# get the number of top edges to select

# this is not sorted by the value of the tensor
num_top_edges = int(0.01 * predict_adj.numel())

value, indices = torch.topk(predict_adj.view(-1), num_top_edges, largest=True)

# convert the 1D indices to 2D indices
indices_2d = torch.stack(torch.unravel_index(indices, predict_adj.shape)).t()

value
print(indices_2d)

In [ ]:
original_edge_index = original_edge_index.t()

In [ ]:
indices_2d[:, 1]

In [ ]:
import numpy as np
entry1 = np.array(data.node_name)[indices_2d[:, 0].numpy()]
entry2 = np.array(data.node_name)[indices_2d[:, 1].numpy()]
# combine the two entries
edges = np.stack([entry1, entry2], axis=1)
edges

In [ ]:
from unipressed import IdMappingClient
request = IdMappingClient.submit(
    source="UniProtKB_AC-ID",
    dest="Gene_Name",
    ids=entry1
)

In [ ]:
# check if first column is 'upQ8NFJ5' and second column is 'P35626'
test = 'up:P16520'
edges[(edges[:, 0] == test) & (edges[:, 1] == 'up:P35626')]
edges[(edges[:, 0] == 'up:P35626') & (edges[:, 1] == test)]

In [ ]:
edges[edges[:, 0] == 'upQ8NFJ5' & edges[:, 1] == 'P35626']

In [ ]:
original_edge_index

In [ ]:
entry1 = np.array(data.node_name)[original_edge_index[:, 0].numpy()]
entry2 = np.array(data.node_name)[original_edge_index[:, 1].numpy()]
# combine the two entries
edges = np.stack([entry1, entry2], axis=1)
edges